## Let's try to get a GPU infos

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May  6 16:34:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off | 00000000:03:00.0  On |                  N/A |
| 20%   32C    P8              10W / 250W |  11114MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Import the libraries

In [2]:
%pip install datasets
%pip install ipywidgets
%pip install accelerate
%pip install sentencepiece
%pip install --upgrade huggingface_hub
%pip -q install git+https://github.com/huggingface/transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from huggingface_hub import login
login()

## Load CSV File

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datasets import load_dataset

data_path = 'exos_to_csv.csv'
dataset = load_dataset('csv', data_files=data_path)

In [5]:
# Split the dataset into 3 sets for train, test and validation
train_testsplit = dataset['train'].train_test_split(test_size=0.2)
test_validsplit = train_testsplit['test'].train_test_split(test_size=0.5)

In [6]:
train_ds = train_testsplit['train']
valid_ds = test_validsplit['train']
test_ds = test_validsplit['test']

In [7]:
train_ds

Dataset({
    features: ['filename', 'consigne', 'enonce'],
    num_rows: 1022
})

In [8]:
valid_ds

Dataset({
    features: ['filename', 'consigne', 'enonce'],
    num_rows: 128
})

In [9]:
test_ds

Dataset({
    features: ['filename', 'consigne', 'enonce'],
    num_rows: 128
})

## Load Mistral Model

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

# With left-padding
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", padding_side="left")

/home/huang_h/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/huang_h/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a pad token by default

### Using HuggingFace Pipeline Method for Text Generation Task

In [13]:
from transformers import pipeline


2024-05-06 16:37:10.849780: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 16:37:10.849840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 16:37:10.856765: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 16:37:10.870392: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 16:37:12.318837: W tensorflow/compiler/tf2

In [51]:
#tokenizer.default_chat_template

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif false == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\\n\\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must

> **Note:**  
The pipeline will take care of all the details of tokenization and calling `apply_chat_template` for you  
So, once the model has a chat template, all you need to do is initialize the pipeline and pass it the list of messages !  

In [14]:
# Iterate through the dataset
def data_iterator(corpus):
    for item in corpus[:5]["consigne"]:

        # Yield a chat-formatted message
        yield {
            "role": "user",
            "content": item
        }

        # formatted_message = tokenizer.apply_chat_template([message], add_generation_prompt=True, tokenize=False)


### Generation Process

In [15]:
import torch

In [16]:
def get_generated_text(corpus, model, tokenizer):

    # Initialize the generator
    generator = pipeline(
        task='text-generation', 
        model=model, 
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        device_map='auto',
        batch_size=2
    )

    # Get the generated text
    messages = data_iterator(corpus)
    generated_text = []

    for msg in messages:
        # Define a prompt
        prompt = f"Générer la consigne en français, éviter les explications, les traductions et limiter la ponctuation spéciale en anglais : {msg['content']}"

        outputs = generator(
            prompt,
            max_new_tokens = 50,
            do_sample=True, 
            temperature=0.7, 
            top_k=50,
            top_p=0.95
        )

        generated_text.append(outputs[0]['generated_text'])

    return generated_text


In [17]:
consigne_generated = get_generated_text(train_ds, model=model, tokenizer=tokenizer)

In [18]:
for sample in consigne_generated:
    print(sample)
    print(100 * "-")

Générer la consigne en français, éviter les explications, les traductions et limiter la ponctuation spéciale en anglais : Recopie ces phrases avec le sujet qui convient pour chaque verbe.

1. The company is making an announcement.
   La compagnie fait une annonce.

2. We are expecting a delivery soon.
   Nous attendons une livraison bientôt.

3
----------------------------------------------------------------------------------------------------
Générer la consigne en français, éviter les explications, les traductions et limiter la ponctuation spéciale en anglais : Écris les questions correspondant aux réponses suivantes.

1. The report states that the company had revenues of $5.2 million last year. What was the company's revenue last year?
2. The sales manager, John, earned a salary of $75,0
----------------------------------------------------------------------------------------------------
Générer la consigne en français, éviter les explications, les traductions et limiter la ponctuati